In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!git clone https://github.com/wl-research/nubia.git
import os
os.chdir('nubia')

Cloning into 'nubia'...
remote: Enumerating objects: 272, done.
remote: Total 272 (delta 0), reused 0 (delta 0), pack-reused 272
Receiving objects: 100% (272/272), 70.09 MiB | 19.31 MiB/s, done.
Resolving deltas: 100% (120/120), done.


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 307kB 6.8MB/s 
     |████████████████████████████████| 184kB 15.9MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 1.1MB 16.4MB/s 
     |████████████████████████████████| 890kB 46.8MB/s 
  Created wheel for fairseq: filename=fairseq-0.9.0-cp36-cp36m-linux_x86_64.whl size=2021167 sha256=ca3eafc5fee3b8362ecb4736f180f8f5a42bb93dd420679beea825120d94e33b
  Stored in directory: /root/.cache/pip/wheels/37/3e/1b/0fa30695dcba41e4b0088067fa40f3328d1e8ee78c22cd4766
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=81078fb1dd40ae24f3401d598c51383a94689683b27b4858c8b33231719200f9
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=340d778fbdf0717c2204e4bbaa804aeec3033023cab49c95346f8ae6720d3c8f
  Stored in directory: /root/.cache/pip

In [ ]:
import pandas as pd
import numpy as np
import string
from string import digits
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, GRU, Embedding, Dense
from keras.models import Model
from nubia import Nubia
import datetime

Using TensorFlow backend.


In [ ]:
nubia = Nubia()

loading archive file pretrained/roBERTa_STS
| [input] dictionary: 50265 types
loading archive file pretrained/roBERTa_MNLI
| dictionary: 50264 types


In [ ]:
lines= pd.read_table('/content/drive/My Drive/rus.txt', names=['eng','rus', '1'])

In [ ]:
lines = lines.drop(['1'], axis=1)
lines = lines[:38696]
lines = shuffle(lines)

In [ ]:
lines.shape

(38696, 2)

In [ ]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.rus=lines.rus.apply(lambda x: x.lower())

# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.rus=lines.rus.apply(lambda x: re.sub("'", '', x))

exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.rus=lines.rus.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng = lines.eng.apply(lambda x: x.translate(remove_digits))
lines.rus = lines.rus.apply(lambda x: x.translate(remove_digits))

# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.rus=lines.rus.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.rus=lines.rus.apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
lines.eng = lines.eng.apply(lambda x : 'START_ '+ x + ' _END')

In [ ]:
lines.sample(10)

,eng,rus
34731,START_ who did you call _END,кому ты звонил
15323,START_ ill just wait _END,я просто подожду
13829,START_ dont struggle _END,не деритесь
10315,START_ ill save tom _END,я спасу тома
19600,START_ dont forget it _END,не забывай об этом
31789,START_ someone screamed _END,ктото завизжал
4907,START_ tom is nuts _END,том не дружит с головой
29000,START_ i like surprises _END,я люблю сюрпризы
32020,START_ thats mine too _END,это тоже моё
28808,START_ i have questions _END,у меня есть вопросы


In [ ]:
# Vocabulary of Russian
all_rus_words=set()
for rus in lines.rus:
    for word in rus.split():
        if word not in all_rus_words:
            all_rus_words.add(word)

# Vocabulary of English 
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

In [ ]:
# Max Length of source sequence
lenght_list=[]
for l in lines.rus:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

10

In [ ]:
# Max Length of target sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

8

In [ ]:
input_words = sorted(list(all_rus_words))
target_words = sorted(list(all_eng_words))
num_encoder_tokens = len(all_rus_words)
num_decoder_tokens = len(all_eng_words)
num_encoder_tokens, num_decoder_tokens

(11582, 4207)

In [ ]:
num_decoder_tokens += 1 # For zero padding
num_encoder_tokens +=1
num_decoder_tokens

4208

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [ ]:
lines.head(10)

,eng,rus
37140,START_ he has curly hair _END,у него кудрявые волосы
31613,START_ rabbits can swim _END,кролики могут плавать
8596,START_ youre weird _END,вы странный
23621,START_ theres a table _END,есть стол
1045,START_ i beg you _END,умоляю вас
33161,START_ tom is very open _END,том очень открыт
19466,START_ do you have one _END,у тебя есть один
9497,START_ i cried again _END,я опять заплакал
33329,START_ tom may not come _END,том возможно не придёт
4885,START_ tom is full _END,том наелся


In [ ]:
#Split the data into train and test
X, y = lines.rus, lines.eng
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((34826,), (3870,))

In [ ]:
#Define a function which generates the data for train and test in batches

def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [ ]:
#Define the encoder-decoder architecture with GRU
latent_dim = 50

In [ ]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_gru = GRU(latent_dim, return_state=True)
encoder_outputs, state_h = encoder_gru(enc_emb)

encoder_states = [state_h]

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_gru = GRU(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_gru(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 32
epochs = 100

In [ ]:
model.load_weights('/content/drive/My Drive/Colab Notebooks/weights_gru_ru-en.h5')

In [ ]:
print('Beginning Training!') 
current_time = datetime.datetime.now() 

model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = 200,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

print("Training took time ", datetime.datetime.now() - current_time)

Beginning Training!


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
200/200 [==============================] - 26s 132ms/step - loss: 1.8450 - acc: 0.4196 - val_loss: 1.7566 - val_acc: 0.4206
Epoch 2/100
200/200 [==============================] - 28s 139ms/step - loss: 1.8038 - acc: 0.4272 - val_loss: 1.6996 - val_acc: 0.4316
Epoch 3/100
200/200 [==============================] - 27s 136ms/step - loss: 1.7942 - acc: 0.4332 - val_loss: 1.7609 - val_acc: 0.4384
Epoch 4/100
200/200 [==============================] - 28s 139ms/step - loss: 1.7749 - acc: 0.4378 - val_loss: 1.7585 - val_acc: 0.4443
Epoch 5/100
200/200 [==============================] - 31s 154ms/step - loss: 1.7663 - acc: 0.4459 - val_loss: 1.7660 - val_acc: 0.4465
Epoch 6/100
200/200 [==============================] - 31s 155ms/step - loss: 1.7229 - acc: 0.4500 - val_loss: 1.6547 - val_acc: 0.4539
Epoch 7/100
200/200 [==============================] - 29s 145ms/step - loss: 1.7030 - acc: 0.4536 - val_loss: 1.7195 - val_acc: 0.4591
Epoch 8/100
200/200 [===========================

In [ ]:
model.save_weights('/content/drive/My Drive/Colab Notebooks/weights_gru_ru-en.h5')

### Inference Setup

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))

decoder_states_inputs = [decoder_state_input_h]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2 = decoder_gru(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
#Finally, generate the output by defining the following function and later calling it.

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h]

    return decoded_sentence

### Evaluation on Train Dataset

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_train[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_train[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_train[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: пожалуйста прекрати это
Actual English translation:  please stop it 
Predicted English translation:  its a lot 
Semantic relation: 1.555252194404602/5.0
Percent chance of contradiction: 1.7395731061697006%
Percent chance of irrelevancy or new information: 66.66330695152283%
Percent chance of logical agreement: 31.59712255001068%

NUBIA score: 0.29142747307207184/1.0


0.29142747307207184

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_train[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_train[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_train[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: стопудово
Actual English translation:  yes of course 
Predicted English translation:  he was up 
Semantic relation: 1.980818510055542/5.0
Percent chance of contradiction: 2.031507156789303%
Percent chance of irrelevancy or new information: 91.29781126976013%
Percent chance of logical agreement: 6.670675426721573%

NUBIA score: 0.2985636073411566/1.0


0.2985636073411566

### Evaluation on Validation Dataset

In [ ]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: мы заблудились
Actual English translation:  we got lost 
Predicted English translation:  were lost 
Semantic relation: 3.7291259765625/5.0
Percent chance of contradiction: 0.26536115910857916%
Percent chance of irrelevancy or new information: 1.0849418118596077%
Percent chance of logical agreement: 98.64969849586487%

NUBIA score: 0.5906952187102021/1.0


0.5906952187102021

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: они доктора
Actual English translation:  theyre doctors 
Predicted English translation:  they are a lot 
Semantic relation: 1.435977816581726/5.0
Percent chance of contradiction: 4.46191169321537%
Percent chance of irrelevancy or new information: 89.06938433647156%
Percent chance of logical agreement: 6.468704342842102%

NUBIA score: 0.32035303384572267/1.0


0.32035303384572267

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: как том мог выиграть
Actual English translation:  how could tom win 
Predicted English translation:  how did tom win 
Semantic relation: 4.144294738769531/5.0
Percent chance of contradiction: 1.8374454230070114%
Percent chance of irrelevancy or new information: 2.0112859085202217%
Percent chance of logical agreement: 96.15126848220825%

NUBIA score: 0.8534810744294751/1.0


0.8534810744294751